## LLave Primaria

In [0]:
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset").strip()

# ✅ Aquí el cambio mínimo y mágico
client = Socrata("www.datos.gov.co", str(token),timeout=1000)


In [0]:
dbutils.widgets.text("codigo_dataset", "")
print("🧪 Widget valor:", repr(dbutils.widgets.get("codigo_dataset")))

In [0]:
client = Socrata("www.datos.gov.co", "KwZhcbkekhw5FpAUWVnzvIdVA", None)
client.get("rpmr-utcd", query="SELECT numero_de_proceso LIMIT 1 OFFSET 0")

In [0]:
import time
# 3. Parámetros de paginación
limit = 500000
offset = 0
write_mode = "overwrite"
reintentos = 5

print(f"Iniciando carga por lotes para el dataset: {codigo_dataset}")

# 4. Bucle para obtener y cargar los datos por lotes
while True:
    intentos = 0

    while intentos < reintentos:
        try:
            print(f"Obteniendo y cargando lote {offset}...")
            # Construye y ejecuta la consulta para el lote actual
            query = f"SELECT numero_del_contrato, numero_de_proceso, nit_de_la_entidad, documento_proveedor, estado_del_proceso LIMIT {limit} OFFSET {offset}"
            results = client.get(codigo_dataset, query=query) 

            # Si la API no devuelve más registros, se termina el script
            if not results:
                print("✅ Carga de datos finalizada. No hay más registros.")
                exit(0)

            # Convierte el lote a un DataFrame de Spark y lo escribe en la tabla Delta
            spark.createDataFrame(results).write \
                .format("delta") \
                .mode(write_mode) \
                .option("overwriteSchema", "true") \
                .saveAsTable("main.diplomado_datos.ids_contratos_procesos")

            print(f"Lote de {len(results)} registros desde offset {offset} cargado.")

            # Se cambia a modo 'append' para las siguientes iteraciones y se incrementa el offset
            write_mode = "append"
            offset += limit
            break

        except Exception as e:
            intentos += 1
            print("Error al obtener o cargar el lote:", e)
            print(f"Intento {intentos} de {reintentos}...")
            time.sleep(20)
    else:
        print(f"❌ No se pudo obtener datos en offset {offset} después de {reintentos} intentos. Asumimos que no hay más registros.")
        print("✅ Carga de datos finalizada con éxito.")
        break

In [0]:
df_secop_id = spark.table("main.diplomado_datos.ids_contratos_procesos")
df_secop_id.count()

In [0]:
# Importar las funciones necesarias de PySpark
from pyspark.sql.functions import sha2, concat_ws, col

# 1. Cargar la tabla correcta desde el catálogo a un DataFrame
# Se asume que esta es la tabla que contiene las columnas que mencionaste.
df_secop_id = spark.table("main.diplomado_datos.ids_contratos_procesos")

# 2. Definir la lista corregida de columnas para el identificador único
columnas_para_hash = [
    "numero_del_contrato",
    "numero_de_proceso",
    "nit_de_la_entidad",
    "documento_proveedor",
    "estado_del_proceso"
]

In [0]:
# 3. Añadir la nueva columna 'id_unico'
# Se concatenan las columnas clave con un separador y se les aplica un hash SHA-2.
df_con_id = df_secop_id.withColumn(
    "id_unico_con_estado",
    sha2(concat_ws("||", *[col(c) for c in columnas_para_hash]), 256)
)

columnas_para_hash_se = [
    "numero_del_contrato",
    "numero_de_proceso",
    "nit_de_la_entidad",
    "documento_proveedor"
]
df_con_id = df_con_id.withColumn(
    "id_unico_sin_estado",
    sha2(concat_ws("||", *[col(c) for c in columnas_para_hash_se]), 256)
)
df_con_id.display()

In [0]:
df_con_id.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("main.diplomado_datos.secop_id_bronze")